In [ ]:
@inproceedings{ghazarian2021plot,
  title={Plot-guided Adversarial Example Construction for Evaluating Open-domain Story Generation},
  author={Sarik Ghazarian and Zixi Liu and Akash S M and Ralph Weischedel and Aram Galstyan and Nanyun Peng},
  booktitle={The 2021 Annual Conference of the North American Chapter of the Association for Computational Linguistics (NAACL)},
  paper_url={https://www.aclweb.org/anthology/2021.naacl-main.343},
  pages={4334–-4344},
  slides_url={https://underline.io/events/122/sessions/4241/lecture/19650-plot-guided-adversarial-example-construction-for-evaluating-open-domain-story-generation},
  year={2021}
}

@inproceedings{Bosselut2019COMETCT,
  title={COMET: Commonsense Transformers for Automatic Knowledge Graph Construction},
  author={Antoine Bosselut and Hannah Rashkin and Maarten Sap and Chaitanya Malaviya and Asli Çelikyilmaz and Yejin Choi},
  booktitle={Proceedings of the 57th Annual Meeting of the Association for Computational Linguistics (ACL)},
  year={2019}
}

@inproceedings{wolf-etal-2020-transformers,
    title = "Transformers: State-of-the-Art Natural Language Processing",
    author = "Thomas Wolf and Lysandre Debut and Victor Sanh and Julien Chaumond and Clement Delangue and Anthony Moi and Pierric Cistac and Tim Rault and Rémi Louf and Morgan Funtowicz and Joe Davison and Sam Shleifer and Patrick von Platen and Clara Ma and Yacine Jernite and Julien Plu and Canwen Xu and Teven Le Scao and Sylvain Gugger and Mariama Drame and Quentin Lhoest and Alexander M. Rush",
    booktitle = "Proceedings of the 2020 Conference on Empirical Methods in Natural Language Processing: System Demonstrations",
    month = oct,
    year = "2020",
    address = "Online",
    publisher = "Association for Computational Linguistics",
    url = "https://www.aclweb.org/anthology/2020.emnlp-demos.6",
    pages = "38--45"
}

#Draft

In [ ]:
'''
			to_modify = modified_plots
			adverb = outer_level[0][1][0]
			noun1,noun2 = outer_level[0][2][0],outer_level[0][2][1]
			output1 = interactive.get_conceptnet_sequence(noun1, self.comet_model, self.sampler, self.data_loader, self.text_encoder, relation)[0]
			output2 = interactive.get_conceptnet_sequence(noun2, self.comet_model, self.sampler, self.data_loader, self.text_encoder, relation)[0]
			manipulation1 = " ".join(noun1,adverb,output2) 
			manipulation2 = " ".join(noun2,adverb,output1) 

			for i1 in modified_plots:
				for i2 in i1:
					for i3 in i2:
			to_modify_flat = []
			for t_m in to_modify: to_modify_flat.extend(t_m)
			
			noun1_idx = to_modify.index(noun1)
			noun2_idx = to_modify.index(noun2)
			
			to_modify[noun1_idx] = " ".join(noun1,adverb,output2)
			to_modify[noun2_idx] = " ".join(noun2,adverb,output1)

			modified_plots[candidate_loc] = "\t".join(to_modify)
			candidates = outer_level'''

In [ ]:
#sys.path.append('/content/drive/My Drive/Plot-guided-Coherence-Evaluation-main/src')
#!python src/main.py --experiment_type conceptnet --experiment_num 0

#from google.colab import files
#src = list(files.upload().values())[0]
#open('storyline_manipulation_ROC.py','wb').write(src)
#import storyline_manipulation_ROC
'''src = list(files.upload().values())[0]
open('src/data/data.py','wb').write(src)
import data as data

src = list(files.upload().values())[0]
open('src/data/config.py','wb').write(src)
import config as cfg

src = list(files.upload().values())[0]
open('storyline_manipulation_ROC.py','wb').write(src)
import functions as interactive'''

#Pip packages

In [1]:
!pip install tensorboardX
!pip install pyinflect
!pip install ftfy==5.1
!pip install fairseq
!pip install transformers==4.12.5
!pip install datasets

     |████████████████████████████████| 51 kB 470 kB/s 
  Created wheel for ftfy: filename=ftfy-5.1-py3-none-any.whl size=39219 sha256=637ef4f92eb2df5d10a3ca521714a9d6f679c24e40be345d4d35504a2565d3a4
  Stored in directory: /root/.cache/pip/wheels/80/0a/4e/64bbd722ea7bed148b8ef1516dbae348d50b296822e7c1ef41
Successfully built ftfy
     |████████████████████████████████| 1.7 MB 12.2 MB/s 
     |████████████████████████████████| 145 kB 70.8 MB/s 
     |████████████████████████████████| 90 kB 11.7 MB/s 
     |████████████████████████████████| 74 kB 4.2 MB/s 
     |████████████████████████████████| 112 kB 75.3 MB/s 
     |████████████████████████████████| 596 kB 61.1 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=c2282d6741ff604e8b7d30e2b43d9660236e6db73f6f598caa44bf510e376c40
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-ru

     |████████████████████████████████| 3.1 MB 12.9 MB/s 
     |████████████████████████████████| 895 kB 38.6 MB/s 
     |████████████████████████████████| 61 kB 665 kB/s 
     |████████████████████████████████| 3.3 MB 55.8 MB/s 
     |████████████████████████████████| 298 kB 13.1 MB/s 
     |████████████████████████████████| 132 kB 75.7 MB/s 
     |████████████████████████████████| 1.1 MB 57.4 MB/s 
     |████████████████████████████████| 243 kB 66.7 MB/s 
     |████████████████████████████████| 160 kB 68.2 MB/s 
     |████████████████████████████████| 271 kB 62.1 MB/s 
     |████████████████████████████████| 192 kB 71.9 MB/s 


In [ ]:
!pip list

#Imports

In [1]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/NLP_Project/')

##### PLOT MANIPULATIONS #############
from base_project.storyline_manipulation_ROC import Plt_manipulations as Plt_manipulations_ROC
from base_project.storyline_manipulation_WP import Plt_manipulations as Plt_manipulations_WP
from new_storyline_manipulation import Plt_manipulations

from scipy.stats import spearmanr, kendalltau
import csv
import json
import torch
import os
import random
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import math
import spacy
import pyinflect
import argparse 
import src.data.data as data
import src.data.config as cfg
import src.interactive.functions as interactive
from fairseq.models.bart import BARTModel
nlp = spacy.load('en_core_web_sm')
np.random.seed(100)

import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import numpy as np
from datasets import load_dataset, load_metric

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#Helper functions

In [ ]:
def separete_prompts():
  data_dir = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Data_/WP/WP_Eval/ManPlts"
  data_dir2 = "/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/WP/WP_ManPlts"  
  prompt_file = open(os.path.join(data_dir, "WP_valid_manipulpltsprompt"), 'r')
  prompts = open(os.path.join(data_dir2, "prompts_valid"), 'w')
  for line in prompt_file.readlines():
    prompt = line.split('<EOL>')[0].strip()
    prompts.write(prompt + '\n')
  prompts.close();prompt_file.close()

def join_prompt_manPlts():
  data_dir = "/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/ROC/ROC_ManPlts"  
  prompts = open(os.path.join(data_dir, "prompts_test"), 'r')
  man_plts = open(os.path.join(data_dir, "Rocstories_test_manipulated_plts"), 'r')
  prompts_plots = open(os.path.join(data_dir, "Rocstories_test_manipulpltsprompt"), 'w')
  prompts_lines,man_plts_lines = prompts.readlines(),man_plts.readlines()
  print(len(prompts_lines),len(man_plts_lines))
  assert len(prompts_lines)==len(man_plts_lines)
  for pr,plt in zip(prompts_lines,man_plts_lines):
    prompt = pr.split("\n")[0].strip()
    plot = plt.split("\n")[0].strip()
    prompts_plots.write(" <EOL> ".join([prompt,plot]) + '\n')
  prompts.close();man_plts.close()
  prompts_plots.close()

def split_ROC_to_source_target():
  data_dir = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Data_/ROC/ROC_Eval"
  file_train = open(os.path.join(data_dir, "Rocstories_train"), 'r')
  lines_tr = file_train.readlines()
  file_val = open(os.path.join(data_dir, "Rocstories_dev"), 'r')
  lines_val = file_val.readlines()
  file_test = open(os.path.join(data_dir, "Rocstories_test"), 'r')
  lines_test = file_test.readlines()

  file_source = open(os.path.join(data_dir, "ROC_train.source"), 'w')
  file_target = open(os.path.join(data_dir, "ROC_train.target"), 'w')
  valid_file_source = open(os.path.join(data_dir, "ROC_valid.source"), 'w')
  valid_file_target = open(os.path.join(data_dir, "ROC_valid.target"), 'w')
  test_file_source = open(os.path.join(data_dir, "ROC_test.source"), 'w')
  test_file_target = open(os.path.join(data_dir, "ROC_test.target"), 'w')

  for line in lines_tr:
    storyplots =  line.split('<EOT>')[1].split('<EOL>')[0].strip()
    file_source.write(storyplots+'\n')
    story_content =  line.split('<EOL>')[1].split('@')[0].strip()
    file_target.write(story_content+'\n')

  for line in lines_val:
    storyplots =  line.split('<EOT>')[1].split('<EOL>')[0].strip()
    valid_file_source.write(storyplots+'\n')
    story_content =  line.split('<EOL>')[1].split('@')[0].strip()
    valid_file_target.write(story_content+'\n')

  for line in lines_test:
    storyplots =  line.split('<EOT>')[1].split('<EOL>')[0].strip()
    test_file_source.write(storyplots+'\n')
    story_content =  line.split('<EOL>')[1].split('@')[0].strip()
    test_file_target.write(story_content+'\n')
  
  file_source.close()
  file_target.close()
  valid_file_source.close()
  valid_file_target.close()
  test_file_source.close()
  test_file_target.close()

In [ ]:
#separete_prompts()
join_prompt_manPlts()
#split_ROC_to_source_target()

2944 2944


#WP

##Step1

In [2]:
#parser = argparse.ArgumentParser()
#parser.add_argument("--COMET_model_file", type=str, default="/content/drive/My Drive/Colab Notebooks/NLP_Project/Plot-guided-Coherence-Evaluation-main/pretrained_models/conceptnet_pretrained_model.pickle")
#parser.add_argument("--device", type=str, default="cpu")
#parser.add_argument("--COMET_sampling_algorithm", type=str, default="beam-10")
#parser.add_argument("--data_dir", type=str, default="Data_/WP/WP_Eval")
#parser.add_argument("--fname", type=str, default="WP_train")
#args = parser.parse_args()
COMET_model_file = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/pretrained_models/conceptnet_pretrained_model.pickle"
device = 0 #torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
COMET_sampling_algorithm = "beam-5"
data_dir = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Data_/WP/WP_Eval"
data_dir2 = "/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/WP"
fname = "WP_valid"#"WP_train"
	
lines_plots = open(os.path.join(data_dir, fname +'.source'), 'r').readlines()#source - plots
lines_stories = open(os.path.join(data_dir, fname+'.target'), 'r').readlines()
assert len(lines_plots)==len(lines_stories)

set_plots = [line.split('\n')[0].strip() for line in lines_plots]
stories = [lines_stories[ind].split('\n')[0].strip() for ind in range(len(lines_plots))]

output_file = fname +'_manipulated_plts'
man_plts_folder = data_dir2+'/WP_ManPlts'
if not os.path.isdir(man_plts_folder):
	os.mkdir(man_plts_folder)
fw_plts = open(os.path.join(man_plts_folder, output_file), 'w')


num_gens= 1	
#plt_changes = Plt_manipulations_WP(COMET_model_file, COMET_sampling_algorithm, device)
print(COMET_model_file)
new_plt_changes = Plt_manipulations(COMET_model_file, COMET_sampling_algorithm, device,ROC_or_WP = False) #False = WP
for ind, story_plots in enumerate(set_plots):
  print('******************{}****************'.format(ind))
  print('STORY PLOTS:\t'+story_plots,"\n")
  print('STORY:\t'+stories[ind],"\n")
  for i in range(num_gens):
    manipulated_story_plts = story_plots
    num_changes = np.random.choice([2,3], size=1, replace=False)[0]	
    ind_technique_apply = np.random.choice([4,5,6,7,8,9], size=num_changes, replace=False)
    print('number of changes {}'.format(num_changes))
    print('the techniques to apply is {}\n'.format(ind_technique_apply))
    for tech_ind in ind_technique_apply:
      '''if tech_ind ==0:
        manipulated_story_plts = plt_changes.insert_antonym(manipulated_story_plts)
        print('after antonym insertion {}'.format(manipulated_story_plts))
      elif tech_ind ==1:
        manipulated_story_plts = plt_changes.repetition(manipulated_story_plts)
        print('after repetitions {}'.format(manipulated_story_plts))
      elif tech_ind ==2:
        manipulated_story_plts = plt_changes.notlogic_ordered(manipulated_story_plts, stories[ind])
        print('after not logically order {}'.format(manipulated_story_plts))
      elif tech_ind ==3:
        manipulated_story_plts = plt_changes.plt_random_insertion(manipulated_story_plts, set_plots)
        print('after random isertion {}'.format(manipulated_story_plts))'''
      if tech_ind ==4:
        manipulated_story_plts = new_plt_changes.insert_antonym_2(manipulated_story_plts)
        print('after antonym insertion 2 \n{}'.format(manipulated_story_plts))
      elif tech_ind ==5:
        manipulated_story_plts = new_plt_changes.contradiction_LogicalReordereing(manipulated_story_plts, stories[ind])
        print('after contradiction_LogicalReordereing {}'.format(manipulated_story_plts))
      elif tech_ind ==6:
        manipulated_story_plts = new_plt_changes.random_deletion(manipulated_story_plts)
        print('after random_deletion:\n{}'.format(manipulated_story_plts))
      elif tech_ind ==7:
        manipulated_story_plts = new_plt_changes.change_location(manipulated_story_plts)
        print('after change locations:\n{}'.format(manipulated_story_plts))
      elif tech_ind ==8:
        manipulated_story_plts = new_plt_changes.property_switching(manipulated_story_plts)
        print('after switch properties:\n{}'.format(manipulated_story_plts))
      elif tech_ind ==9:
        manipulated_story_plts = new_plt_changes.not_capable_of_prerequisite(manipulated_story_plts)
        print('after switch properties:\n{}'.format(manipulated_story_plts))
    #print(manipulated_story_plts)
    fw_plts.write(manipulated_story_plts.strip() + '\n')
  print('_________________________________________')
fw_plts.close()

/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/pretrained_models/conceptnet_pretrained_model.pickle
/content/drive/My Drive/Colab Notebooks/NLP_Project/data/conceptnet/processed/generation/rel_language-trainsize_100-devversion_12-maxe1_10-maxe2_15-maxr_5.pickle
Loading data from: /content/drive/My Drive/Colab Notebooks/NLP_Project/data/conceptnet/processed/generation/rel_language-trainsize_100-devversion_12-maxe1_10-maxe2_15-maxr_5.pickle
31
******************0****************
STORY PLOTS:	answering	machine	</s>	</s>	you	know	our office	received	a crank call	from this number	playing	</s>	sight	corrected	dial	a specific number	</s>	want	to see anymore	you	ought	to look into that	</s>	you	call	we	recommend	some trauma specialists	</s>	Maria	sighed	</s>	fix	six years of disability 

STORY:	Maria hit the voicemail button on her answering machine . </s> `` Mrs. Tenenfield , this is Dr. Routabagh calling . </s> I wanted to let you know our office received a crank call from 

/content/drive/My Drive/Colab Notebooks/NLP_Project/src/evaluate/sampler.py:275: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  for i in beam_losses]
/content/drive/My Drive/Colab Notebooks/NLP_Project/src/evaluate/sampler.py:276: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ended = ended.index_select(0, top_beam_idxs // self.opt.eval

Streaming output truncated to the last 5000 lines.


Input Entity:    beat his wife
Target Relation: HasPrerequisite

Candidate Sequences:
have sex with her
have sex
get knife
slap her
get weapon


Input Entity:    rest
Target Relation: CapableOf

Candidate Sequences:
help you live long
help reduce stress
help you stay awake
be good thing
help you sleep


Input Entity:    help you live long
Target Relation: HasPrerequisite

Candidate Sequences:
work hard
get job
be patient
work
effort


Input Entity:    clean
Target Relation: CapableOf

Candidate Sequences:
smell bad
smell good
smell nice
look good on you
look good


Input Entity:    smell bad
Target Relation: HasPrerequisite

Candidate Sequences:
take shower
go to bathroom
shower
have shower
get shower


after switch properties:
fell	wept	</s>	 	</s>	So many lives	lost	wept	</s>	been	in vain	</s>	fallen	comrades	fallen comrades	died	</s>	Blood can not blood	seeped	hand	laid	his abdomen	</s>	knew can not be careful	this one was fatal	<

##Step 2 (Fine-tuning BART) - done

##Step 3 (Generate implausible stories with BART)

In [ ]:
'''parser = argparse.ArgumentParser()
parser.add_argument("--bart_model_path", type=str, default="Models/Ft_BART_Story_Generator/WP/", help="model path including finetuned BART model as the conditional LM")
parser.add_argument("--data_path", type=str, default='Data/WP/WP_Eval/', help='data path for WP_Eval data')
parser.add_argument("--json_file", type=str, default='WP_AF_input.json', help='json input files')
parser.add_argument("--num_negative_samples", type=int, default=6, help="number of negative (implausible) samples to generate for each plausible story")
parser.add_argument("--batch_size", type=int, default=20, help='batch size to generate samples')
parser.add_argument("--max_len_b", type=int, default=750, help='max length of stories')
parser.add_argument("--min_len", type=int, default=10, help='min length of stories')
parser.add_argument("--sampling_topk", type=int, default=50, help='topk sampling')
parser.add_argument("--temperature", type=float, default=0.8, help='temperature value')
parser.add_argument("--beam", type=float, default=4, help='beam size')'''

bart_model_path = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Models/Ft_BART_Story_Generator/WP/"
data_path = "/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/WP/WP_ManPlts/"
data_path2 = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Data_/WP/WP_Eval/"
json_file = "WP_AF_input.json"
num_negative_samples = 6
batch_size = 20
max_len_b = 750
min_len = 10
sampling_topk = 50
temperature = 0.8
beam = 4
#args = {"bart_model_path":bart_model_path,"data_path":data_path,"json_file":json_file,
#        "num_negative_samples":num_negative_samples,"batch_size":batch_size,
#        "max_len_b":max_len_b,"min_len":min_len,"sampling_topk":sampling_topk,
#        "temperature":temperature,"beam":beam}

bart = BARTModel.from_pretrained(bart_model_path,
                                 checkpoint_file='checkpoint_best.pt',
                                 data_name_or_path='/content/drive/My Drive/Colab Notebooks/NLP_Project/wp_kw_story-bin')
bart.cuda()
bart.eval()
bart.half()

In [ ]:
#af = AF_Data_Generation_WP(bart_model_path)
for  file_type in ['train', 'valid']:
  lines_gt_stories = open(data_path2+'WP_{}.target'.format(file_type), 'r').readlines()
  manipulated_plots = open(data_path+'WP_{}_manipulated_plts'.format(file_type), 'r').readlines()
  lines_plots_prompts = open(data_path+'WP_{}_manipulpltsprompt'.format(file_type), 'r').readlines()
  plots_prmpts={}
  manipulated_plots_new,prompts = [],[]
  gt_stories,num_sents_gt_stories=[],[]
  for line in lines_plots_prompts:
  	plts = line.split('\n')[0].split('<EOL>')[1].strip()
  	if plts not in plots_prmpts:
  		plots_prmpts[plts] = line.split('<EOL>')[0].strip()
  for ind, plt in enumerate(manipulated_plots[:5]):
  	plt  = plt.split('\n')[0]
  	if plt in plots_prmpts.keys():
  		manipulated_plots_new.append(plt)
  		prompts.append(plots_prmpts[plt])#add two of manipulated plts' prompt (used in create_json_AF_input function)
  		prompts.append(plots_prmpts[plt])
  for ind_line, story in  enumerate(lines_gt_stories):
  	gt_story = lines_gt_stories[ind_line].split('\n')[0]
  	gt_stories.append(gt_story)
  	num_sents_gt_stories.append(len(gt_story.split('</s>')))
   
  for ind in range(num_negative_samples):  #Generate args.num_negative_samples different tsv files each including gt_stories as positive and generated negative stories as implausible ones
    #af.generate_implausible_stories(args, gt_stories, manipulated_plots_new, num_sents_gt_stories, ind,file_type) 	def generate_implausible_stories(self, args, gt_stories, manipulated_plots, num_sents_gt_stories, ind_file,file_type):
    file_ = open(os.path.join(data_path, '{}_pos_neg_stories_{}.tsv'.format(file_type, ind)), 'w')
    tsv_writer = csv.writer(file_, delimiter='\t', lineterminator='\n')
    count,indj = 1,0
    conv_lines,gt_stories_lines,num_sents_gt_lines = [],[],[]
    for ind_line, plots in  enumerate(manipulated_plots_new):
      plots = plots.strip()
      conv_lines.append(plots)
      gt_stories_lines.append(gt_stories[ind_line])
      num_sents_gt_lines.append(num_sents_gt_stories[ind_line])
      if count % batch_size == 0:
        with torch.no_grad():
          hyps = bart.sample(conv_lines, sampling =True, lenpen=2.0, max_len_b=max_len_b, min_len=min_len, sampling_topk=sampling_topk, temperature=temperature, beam=beam)	
          print("HYPS: ",hyps)
        for i, hypothesis in enumerate(hyps):
          hypothesis = hypothesis.strip().split('</s>')[:num_sents_gt_lines[i]]
          hypothesis = '</s>'.join(hypothesis).strip()
          if hypothesis[-1] not in ['.', '!', '?', '...', '?!']:
            hypothesis = hypothesis[:hypothesis.rfind('</s>')]
          tsv_writer.writerow([indj, '1', indj, gt_stories_lines[i]]);indj+=1
          tsv_writer.writerow([indj, '0', indj, hypothesis]);indj+=1	
        num_sents_gt_lines,gt_stories_lines,conv_lines=[],[],[]
      count +=1
    if conv_lines != []: #if finished before last batch reaches 20
      print("conv_lines: ",conv_lines)
      hyps = bart.sample(conv_lines, sampling=True, lenpen=2.0, max_len_b=max_len_b, min_len=min_len, sampling_topk=sampling_topk, temperature=temperature, beam=beam)
      print("HYPS: ",ind,"  ",hyps)
      for i, hypothesis in enumerate(hyps):
        hypothesis = hypothesis.strip().split('</s>')[:num_sents_gt_lines[i]]
        hypothesis = '</s>'.join(hypothesis)
        if hypothesis[-1] not in ['.', '!', '?', '...', '?!']:
          hypothesis = hypothesis[:hypothesis.rfind('</s>')]
        tsv_writer.writerow([indj, '1', indj, gt_stories_lines[i]]);indj+=1
        tsv_writer.writerow([indj, '0', indj, hypothesis]);indj+=1
    file_.close()

In [ ]:
#af.create_json_AF_input(args, prompts),def create_json_AF_input(self, args, prompts):
af_input_dict = {}
for mode in ['train', 'valid']:
  for ind in range(num_negative_samples):
    af_input_dict['fr_{}_{}'.format(mode,ind)] = open(os.path.join(data_path, '{}_pos_neg_stories_{}.tsv'.format(mode, ind)), 'r')
    af_input_dict['lines_{}_{}'.format(mode,ind)] = af_input_dict['fr_{}_{}'.format(mode,ind)].readlines()

fw = open(os.path.join(data_path, json_file), 'w')
output = []
for mode in ['train', 'valid']:
  ind=0
  for i in range(0, len(af_input_dict['lines_{}_{}'.format(mode,ind)])-3, 2):
    output_text ={}
    gt_story = af_input_dict['lines_{}_{}'.format(mode,ind)][i].split('\t')[3].split('\n')[0].strip()
    gens=[]
    for ind in range(num_negative_samples):
      print(i+1,len(af_input_dict['lines_{}_{}'.format(mode,ind)]))
      if(len(af_input_dict['lines_{}_{}'.format(mode,ind)])==0):print(mode,ind,af_input_dict['lines_{}_{}'.format(mode,ind)])
      af_input_dict['gen_story{}'.format(ind)]=af_input_dict['lines_{}_{}'.format(mode,ind)][i+1].split('\t')[3].split('\n')[0].strip()
      gens.append(af_input_dict['gen_story{}'.format(ind)])
    output_text["ctx"] = prompts[i]
    output_text["gt_detok"] = gt_story
    output_text["gens"] = gens 
    output.append(output_text)
    ind=0		
fw.write('\n'.join(json.dumps(i, ensure_ascii=False) for i in output))

##Step 4 (convert to csv)

In [ ]:
#parser = argparse.ArgumentParser()
#parser.add_argument("--data_path", type=str, default='Data/WP/WP_Eval/', help='data path for WP_Eval data')
#parser.add_argument("--output_AF", type=str, default='AF_output.json', help='The result file from AF')
#parser.add_argument("--train_output", type=str, default='AF_ManPlts_train.tsv', help='The training file resulted from AF applied on WP manipulated plots to be used for training the evaluator')
#parser.add_argument("--valid_output", type=str, default='AF_ManPlts_valid.tsv', help='The validation file resulted from AF applied on WP manipulated plots to be used for validating the evaluator')
#parser.add_argument("--test_output", type=str, default='AF_ManPlts_test.tsv', help='The testing file resulted from AF applied on WP manipulated plots to be used for testing the evaluator')
#args = parser.parse_args()
#data_path = '/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Data_/WP/WP_Eval/'
data_path = '/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/WP/WP_ManPlts/'
data_path2 = '/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/WP/WP_AF_ManPlts/'
output_AF = "WP_AF_input.json"#'AF_output.json'

fr = open(os.path.join(data_path, output_AF), 'r')
fw_train = open(os.path.join(data_path2, 'AF_ManPlts_train.tsv'), 'w')
fw_valid = open(os.path.join(data_path2, 'AF_ManPlts_valid.tsv'), 'w')
fw_test = open(os.path.join(data_path2, 'AF_ManPlts_test.tsv'), 'w')
tsv_train = csv.writer(fw_train, delimiter='\t', lineterminator='\n')
tsv_valid = csv.writer(fw_valid, delimiter='\t', lineterminator='\n')
tsv_test = csv.writer(fw_test, delimiter='\t', lineterminator='\n')

list_ctx,list_gt,list_gens = [],[],[]
num_stories= 0
for line in fr:
  line = json.loads(line)
  list_ctx.append(line['ctx'])
  list_gt.append(line['gt_detok'])
  gens = []
  for l in line['gens']:
    gens.append(l)
  list_gens.append(gens)
	#sel_inds = line['assignment'][-1] #line['assignment'][-1] shows the index of most challenging generated stories based on the applied AF
	#for ind in sel_inds:
  #	gens.append(line['gens'][ind])

num_stories=len(list_ctx)
num_train_stories = math.ceil((60*num_stories)/100)
num_valid_stories = math.ceil((20*num_stories)/100)
num_test_stories = math.ceil((20*num_stories)/100)
tsv_out = {}
start_ind={'train':0, 'valid':num_train_stories, 'test': num_train_stories + num_valid_stories}
for mode in ['train', 'valid', 'test']:
	st_ind = start_ind['{}'.format(mode)]
	start_ind['{}_ctx'.format(mode)] = list_ctx[st_ind:st_ind+eval('num_{}_stories'.format(mode))]
	start_ind['{}_gt'.format(mode)] = list_gt[st_ind:st_ind+eval('num_{}_stories'.format(mode))]
	start_ind['{}_gens'.format(mode)] = list_gens[st_ind:st_ind+eval('num_{}_stories'.format(mode))]

for mode in ['train', 'valid', 'test']:
  line_ind = 0
  print(start_ind['{}_gt'.format(mode)])
  for ind, gt in enumerate(start_ind['{}_gt'.format(mode)]):
    print("cc",mode)
    eval('tsv_{}'.format(mode)).writerow([line_ind, 1, line_ind , gt])
    line_ind+=1
    for gen in start_ind['{}_gens'.format(mode)][ind]:
      eval('tsv_{}'.format(mode)).writerow([line_ind, 0, line_ind, gen])
      line_ind+=1

['I have two trees out in the yard that I always sit under and take part in a poorly recreated *hanami* . </s> The blossoms float and drift on the wind . </s> Ephemeral little creatures they are . </s> Beautiful ephemeral creatures that scatter their beauty all over the neighboring lawns and houses . </s> A good glass of alcohol always goes with it . </s> Red blossoms are painted all over my study and I find that it ’ s just as beautiful . </s> Maybe I should see if I can find a red cherry blossom tree . </s> I should put a footrest out in the front yard next spring too . </s> It ’ s so much more comfortable to relax with a footrest to watch the blossoms .', "Gabriel stumbled and twirled away from the door as it closed behind him , swallowing up the music and the voices . </s> The air was cool and crisp and purple in the dying streetlights . </s> He rubbed his eyes and tried to remember where the subway was . </s> Left or right ? Right ? Right , right ? He drew a deep breath , turned t

##Step 5 (run_glue , Evaluators)

##Step 6 (Get Scores)

#.

#.

#.

#ROC

##Step 1 (plot manipulations)

In [ ]:
COMET_model_file = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/pretrained_models/conceptnet_pretrained_model.pickle"
device = 0 #0 #torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
COMET_sampling_algorithm = "beam-1"
data_dir = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Data_/ROC/ROC_Eval"
data_dir2 = "/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/ROC"
fname = "Rocstories_train" #"Rocstories_train" "Rocstories_test"
#prompts = open(os.path.join(data_dir2, "ROC_ManPlts/prompts_train"), 'w') #"prompts_valid" "prompts_test"
#file_source = open(os.path.join(data_dir, "Roc_train.source"), 'w')
#file_target = open(os.path.join(data_dir, "Roc_train.target"), 'w')

fr_stories = open(os.path.join(data_dir, fname), 'r')
lines = fr_stories.readlines()
stories = []
set_plots =[]

for line in lines:
	prompt = line.split('<EOT>')[0]
	#prompts.write(prompt + '\n')
	storyplots =  line.split('<EOT>')[1].split('<EOL>')[0].strip()
	#file_source.write(storyplots+'\n')
	story_content =  line.split('<EOL>')[1].split('@')[0].strip()
	#file_target.write(story_content+'\n')
	stories.append(story_content)
	set_plots.append(storyplots)
prompts.close()
		

output_file = fname +'_manipulated_plts_video'
man_plts_folder = data_dir2+'/ROC_ManPlts'
if not os.path.isdir(man_plts_folder):
	os.mkdir(man_plts_folder)
fw_plts = open(os.path.join(man_plts_folder, output_file), 'w')
	
num_gens= 1	
new_plt_changes = Plt_manipulations(COMET_model_file, COMET_sampling_algorithm, device,ROC_or_WP = True) #True=ROC,False=WP
for ind, story_plots in enumerate(set_plots):
	print('******************{}****************'.format(ind))
	print('STORY PLOTS:\t'+story_plots,"\n")
	print('STORY:\t'+stories[ind],"\n")
	for i in range(num_gens):
		manipulated_story_plts = story_plots
		num_changes = np.random.choice([2,3], size=1, replace=False)[0]	
		ind_technique_apply = np.random.choice([4,5,6,7,8,9], size=num_changes, replace=False) #
		print('number of changes {}'.format(num_changes))
		print('the techniques to apply is {}\n'.format(ind_technique_apply))
		for tech_ind in ind_technique_apply:
			'''if tech_ind ==0:
				manipulated_story_plts = plt_changes.insert_antonym(manipulated_story_plts)
				print('after antonym insertion {}'.format(manipulated_story_plts))
			elif tech_ind ==1:
				manipulated_story_plts = plt_changes.repetition(manipulated_story_plts)
				print('after repetitions {}'.format(manipulated_story_plts))
			elif tech_ind ==2:
				manipulated_story_plts = plt_changes.notlogic_ordered(manipulated_story_plts, stories[ind])
				print('after not logically order {}'.format(manipulated_story_plts))
			elif tech_ind ==3:
				manipulated_story_plts = plt_changes.plt_random_insertion(manipulated_story_plts, set_plots)
				print('after random isertion {}'.format(manipulated_story_plts))'''
			if tech_ind ==4:
				manipulated_story_plts = new_plt_changes.insert_antonym_2(manipulated_story_plts)
				print('after antonym insertion 2 \n{}'.format(manipulated_story_plts))
			elif tech_ind ==5:
				manipulated_story_plts = new_plt_changes.contradiction_LogicalReordereing(manipulated_story_plts, stories[ind])
				print('after contradiction_LogicalReordereing {}'.format(manipulated_story_plts))
			elif tech_ind ==6:
				manipulated_story_plts = new_plt_changes.random_deletion(manipulated_story_plts)
				print('after random_deletion:\n{}'.format(manipulated_story_plts))
			elif tech_ind ==7:
				manipulated_story_plts = new_plt_changes.change_location(manipulated_story_plts)
				print('after change locations:\n{}'.format(manipulated_story_plts))
			elif tech_ind ==8:
				manipulated_story_plts = new_plt_changes.property_switching(manipulated_story_plts)
				print('after switch properties:\n{}'.format(manipulated_story_plts))
			elif tech_ind ==9:
				manipulated_story_plts = new_plt_changes.not_capable_of_prerequisite(manipulated_story_plts)
				print('after switch properties:\n{}'.format(manipulated_story_plts))
		#print(manipulated_story_plts)
		fw_plts.write(manipulated_story_plts.strip() + '\n')
	print('_________________________________________')
fw_plts.close()

Streaming output truncated to the last 5000 lines.
number of changes 2
the techniques to apply is [4 9]

after antonym insertion 2 
purchased	brand	local dealership	#	drove	started	#determined	music	coming	engine#	determined	noise	coming	engine	#determined	music	coming	engine#	dealership	explained	problem	#	replaced	faulty engine
[('brand', 0, 1, 0), ('dealership', 0, 2, 1), ('drove', 1, 0, 0), ('music', 2, 1, 0), ('engine', 2, 3, 0), ('noise', 3, 1, 0), ('engine', 3, 3, 0), ('music', 4, 1, 0), ('engine', 4, 3, 0), ('dealership', 5, 0, 0), ('problem', 5, 2, 0), ('faulty', 6, 1, 0)]
Input Entity:    faulty
Target Relation: CapableOf

Candidate Sequences:
cause injury


Input Entity:    cause injury
Target Relation: HasPrerequisite

Candidate Sequences:
think


Input Entity:    faulty
Target Relation: CapableOf

Candidate Sequences:
cause injury


Input Entity:    cause injury
Target Relation: HasPrerequisite

Candidate Sequences:
think


after switch properties:
purchased	brand	local de

##Step 2 (BART tuning) done

##Step 3

In [ ]:
bart_model_path = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Models/Ft_BART_Story_Generator/ROC/"
data_path = "/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/ROC/ROC_ManPlts/"
data_path2 = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Data_/ROC/ROC_Eval/"
json_file = "ROC_AF_input.json"
num_negative_samples = 6

In [ ]:
batch_size = 20
max_len_b = 750
min_len = 10
sampling_topk = 50
temperature = 0.8
beam = 4

bart = BARTModel.from_pretrained(bart_model_path,
                                 checkpoint_file='checkpoint_best.pt',
                                 data_name_or_path='/content/drive/My Drive/Colab Notebooks/NLP_Project/roc_kw_story-bin')
bart.cuda()
bart.eval()
bart.half()


In [ ]:
for  file_type in ['train', 'valid','test']:   
  lines_gt_stories = open(data_path2+'ROC_{}.target'.format(file_type), 'r').readlines()
  manipulated_plots = open(data_path+'Rocstories_{}_manipulated_plts'.format(file_type), 'r').readlines()
  lines_plots_prompts = open(data_path+'Rocstories_{}_manipulpltsprompt'.format(file_type), 'r').readlines()
  plots_prmpts={}
  manipulated_plots_new,prompts = [],[]
  gt_stories,num_sents_gt_stories=[],[]
  for line in lines_plots_prompts:
  	plts = line.split('\n')[0].split('<EOL>')[1].strip()
  	if plts not in plots_prmpts:
  		plots_prmpts[plts] = line.split('<EOL>')[0].strip()
  for ind, plt in enumerate(manipulated_plots): #[:5]
  	plt  = plt.split('\n')[0]
  	if plt in plots_prmpts.keys():
  		manipulated_plots_new.append(plt)
  		prompts.append(plots_prmpts[plt])#add two of manipulated plts' prompt (used in create_json_AF_input function)
  		prompts.append(plots_prmpts[plt])
  for ind_line, story in  enumerate(lines_gt_stories):
  	gt_story = lines_gt_stories[ind_line].split('\n')[0]
  	gt_stories.append(gt_story)
  	num_sents_gt_stories.append(len(gt_story.split('</s>')))
 
  for ind in range(num_negative_samples):  #Generate args.num_negative_samples different tsv files each including gt_stories as positive and generated negative stories as implausible ones
    #af.generate_implausible_stories(args, gt_stories, manipulated_plots_new, num_sents_gt_stories, ind,file_type) 	def generate_implausible_stories(self, args, gt_stories, manipulated_plots, num_sents_gt_stories, ind_file,file_type):
    file_ = open(os.path.join(data_path, '{}_pos_neg_stories_{}.tsv'.format(file_type, ind)), 'w')
    tsv_writer = csv.writer(file_, delimiter='\t', lineterminator='\n')
    count,indj = 1,0
    conv_lines,gt_stories_lines,num_sents_gt_lines = [],[],[]
    for ind_line, plots in  enumerate(manipulated_plots_new):
      plots = plots.strip()
      conv_lines.append(plots)
      gt_stories_lines.append(gt_stories[ind_line])
      num_sents_gt_lines.append(num_sents_gt_stories[ind_line])
      if count % batch_size == 0:
        with torch.no_grad():
          hyps = bart.sample(conv_lines, sampling =True, lenpen=2.0, max_len_b=max_len_b, min_len=min_len, sampling_topk=sampling_topk, temperature=temperature, beam=beam)	
          print("HYPS: ",hyps)
        for i, hypothesis in enumerate(hyps):
          hypothesis = hypothesis.strip().split('</s>')[:num_sents_gt_lines[i]]
          hypothesis = '</s>'.join(hypothesis)
          hypothesis = hypothesis.replace('  ', ' ')
          #if hypothesis[-1] not in ['.', '!', '?', '...', '?!']:
          #  hypothesis = hypothesis[:hypothesis.rfind('</s>')]
          tsv_writer.writerow([indj, '1', indj, gt_stories_lines[i]]);indj+=1
          tsv_writer.writerow([indj, '0', indj, hypothesis]);indj+=1	
        num_sents_gt_lines,gt_stories_lines,conv_lines=[],[],[]
      count +=1
    if conv_lines != []: #if finished before last batch reaches 20
      #print("conv_lines: ",conv_lines)
      hyps = bart.sample(conv_lines, sampling=True, lenpen=2.0, max_len_b=max_len_b, min_len=min_len, sampling_topk=sampling_topk, temperature=temperature, beam=beam)
      #print("HYPS: ",ind,"  ",hyps)
      for i, hypothesis in enumerate(hyps):
        hypothesis = hypothesis.strip().split('</s>')[:num_sents_gt_lines[i]]
        hypothesis = '</s>'.join(hypothesis)
        hypothesis = hypothesis.replace('  ', ' ')
        #if hypothesis[-1] not in ['.', '!', '?', '...', '?!']:
        #  hypothesis = hypothesis[:hypothesis.rfind('</s>')]
        tsv_writer.writerow([indj, '1', indj, gt_stories_lines[i]]);indj+=1
        tsv_writer.writerow([indj, '0', indj, hypothesis]);indj+=1
    file_.close()

HYPS:  ['the cat smelled the urine . </s> it was curious about flower cat . </s> the cat urinated . </s> flower cat ran off to get a loan from the debtor . </s> the creditor was surprised that the cat was soaked in urine . </s> the debtor was curious about flower cat . </s> the debtor saw the cat and urinated on the cat . </s> the owner took the cat to the vet . </s> the vet was surprised that the cat was soaked in urine .', 'the boy went on a date . </s> he was nervous about dinner . </s> he was a little bit too long . </s> the girl was a little too short . </s> the date was long . </s> when he arrived at the dinner he was very shy . </s> he was nervous about dinner . </s> she was a little too short . </s> he did not even sit on her date . </s> he threw his arms up and called her a cute girl .', "heather was a wife of a bank manager . </s> her husband was very disorganized . </s> she made him get a new job at a bank in order to work . </s> he worked the job , moving to a new city . </

In [ ]:
prompts = []
for  file_type in ['train', 'valid','test']:   
  manipulated_plots = open(data_path+'Rocstories_{}_manipulated_plts'.format(file_type), 'r').readlines()
  lines_plots_prompts = open(data_path+'Rocstories_{}_manipulpltsprompt'.format(file_type), 'r').readlines()
  plots_prmpts={}
  gt_stories,num_sents_gt_stories=[],[]
  for line in lines_plots_prompts:
  	plts = line.split('\n')[0].split('<EOL>')[1].strip()
  	if plts not in plots_prmpts:
  		plots_prmpts[plts] = line.split('<EOL>')[0].strip()
  for ind, plt in enumerate(manipulated_plots): #[:5]
  	plt  = plt.split('\n')[0]
  	if plt in plots_prmpts.keys():
  		prompts.append(plots_prmpts[plt])#add two of manipulated plts' prompt (used in create_json_AF_input function)
  		prompts.append(plots_prmpts[plt])
    
af_input_dict = {}
for mode in ['train', 'valid','test']:
  for ind in range(num_negative_samples):
    af_input_dict['fr_{}_{}'.format(mode,ind)] = open(os.path.join(data_path, '{}_pos_neg_stories_{}.tsv'.format(mode, ind)), 'r')
    af_input_dict['lines_{}_{}'.format(mode,ind)] = af_input_dict['fr_{}_{}'.format(mode,ind)].readlines()

fw = open(os.path.join(data_path, json_file), 'w')
output = []
for mode in ['train', 'valid','test']:
  ind=0
  for i in range(0, len(af_input_dict['lines_{}_{}'.format(mode,ind)]), 2): #thru lines_train_0
    output_text ={}
    gt_story = af_input_dict['lines_{}_{}'.format(mode,ind)][i].split('\t')[3].split('\n')[0].strip()
    gens=[]
    #print(len(prompts))
    for ind in range(num_negative_samples):
      #print(i+1,len(af_input_dict['lines_{}_{}'.format(mode,ind)]))
      #if(len(af_input_dict['lines_{}_{}'.format(mode,ind)])==0):print(mode,ind,af_input_dict['lines_{}_{}'.format(mode,ind)])
      af_input_dict['gen_story{}'.format(ind)]=af_input_dict['lines_{}_{}'.format(mode,ind)][i+1].split('\t')[3].split('\n')[0].strip()
      gens.append(af_input_dict['gen_story{}'.format(ind)])
    output_text["ctx"] = prompts[i]
    output_text["gt_detok"] = gt_story
    output_text["gens"] = gens 
    output.append(output_text)
    ind = 0	

fw.write('\n'.join(json.dumps(i, ensure_ascii=False) for i in output))
fw.close()

##Step 4 (convert to tsv)

In [ ]:
#parser = argparse.ArgumentParser()
#parser.add_argument("--data_path", type=str, default='Data/WP/WP_Eval/', help='data path for WP_Eval data')
#parser.add_argument("--output_AF", type=str, default='AF_output.json', help='The result file from AF')
#parser.add_argument("--train_output", type=str, default='AF_ManPlts_train.tsv', help='The training file resulted from AF applied on WP manipulated plots to be used for training the evaluator')
#parser.add_argument("--valid_output", type=str, default='AF_ManPlts_valid.tsv', help='The validation file resulted from AF applied on WP manipulated plots to be used for validating the evaluator')
#parser.add_argument("--test_output", type=str, default='AF_ManPlts_test.tsv', help='The testing file resulted from AF applied on WP manipulated plots to be used for testing the evaluator')
#args = parser.parse_args()
data_path = '/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/ROC/'
output_AF = 'ROC_ManPlts/ROC_AF_input.json'
train_output = 'AF_ManPlts_train.csv'
valid_output = 'AF_ManPlts_valid.csv'
test_output = 'AF_ManPlts_test.csv'

fr = open(os.path.join(data_path, output_AF), 'r')
fw_train = open(os.path.join(data_path, 'ROC_AF_ManPlts/'+train_output), 'w')
fw_valid = open(os.path.join(data_path, 'ROC_AF_ManPlts/'+valid_output), 'w')
fw_test = open(os.path.join(data_path, 'ROC_AF_ManPlts/'+test_output), 'w')
csv_train = csv.writer(fw_train, delimiter=',', lineterminator='\n')
csv_valid = csv.writer(fw_valid, delimiter=',', lineterminator='\n')
csv_test = csv.writer(fw_test, delimiter=',', lineterminator='\n')

list_ctx,list_gt,list_gens = [],[],[]
num_stories= 0
for line in fr:
	line = json.loads(line)
	list_ctx.append(line['ctx'])
	list_gt.append(line['gt_detok'])
	gens = []
	for l in line['gens'][:3]:
		gens.append(l)
	list_gens.append(gens)
 

num_stories=len(list_ctx)
num_train_stories = math.ceil((60*num_stories)/100)
num_valid_stories = math.ceil((20*num_stories)/100)
num_test_stories = math.ceil((20*num_stories)/100)
tsv_out = {}
start_ind={'train':0, 'valid':num_train_stories, 'test': num_train_stories + num_valid_stories}
for mode in ['train', 'valid', 'test']:
	st_ind = start_ind['{}'.format(mode)]
	start_ind['{}_ctx'.format(mode)] = list_ctx[st_ind:st_ind+eval('num_{}_stories'.format(mode))]
	start_ind['{}_gt'.format(mode)] = list_gt[st_ind:st_ind+eval('num_{}_stories'.format(mode))]
	start_ind['{}_gens'.format(mode)] = list_gens[st_ind:st_ind+eval('num_{}_stories'.format(mode))]

for mode in ['train', 'valid', 'test']:
  line_ind = 0
  #print(start_ind['{}_gt'.format(mode)])
  for ind, gt in enumerate(start_ind['{}_gt'.format(mode)]):
    #print("cc",mode)
    eval('csv_{}'.format(mode)).writerow([ 1 , gt])
    line_ind+=1
    for gen in start_ind['{}_gens'.format(mode)][ind]:
      eval('csv_{}'.format(mode)).writerow([0, gen])
      line_ind+=1
fw_train.close()
fw_valid.close()
fw_test.close()

## Step 5 (feed tsvs to evaluators)

##Step 6 (get scores)

In [ ]:
#parser = argparse.ArgumentParser()
#parser.add_argument("--data_path", type=str, default="Data/AMT/")
#parser.add_argument("--data", type=str, default="WP or ROC")
#args = parser.parse_args()
data_path = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Data_/AMT"
data = "WP"#"ROC"
	
if data=='WP':
	fname_human_scores='AMT_WP.CSV'
	fname_model_scores='Pred_advWP_longformer.txt'
	max_stoies_in_hit=2
else:
	fname_human_scores='AMT_ROC.CSV'
	fname_model_scores='Pred_advROC_roberta.txt'
	max_stoies_in_hit=5

#SAME FOR LONGFORMER AND ROBERTA (300 lines)
fr_preds = open(os.path.join(data_path, fname_model_scores), 'r')
preds=fr_preds.readlines()
preds_scores={}
for line in preds:
	text, score = line.split('\t')
	text = text.strip()
	score = float(score.split('\n')[0])
	preds_scores[text] = score

csv_ifile = open(os.path.join(data_path,fname_human_scores), 'r')
csv_reader = csv.DictReader(csv_ifile)
set_proc_texts = []
proc_texts_scores={} 
for row in csv_reader:
  for i in range(1, max_stoies_in_hit+1): #twice for WP/ 5 times for ROC
    key = 'Input.text'+str(i)
    sent = row[key].split('\n')[0].strip()
    #row[key] = sent
    score =row['Answer.text'+str(i)]
    if sent not in set_proc_texts:
      set_proc_texts.append(sent)
      proc_texts_scores[sent] = str(score)
    else:
      proc_texts_scores[sent] = proc_texts_scores[sent] + ' ' +str(score)

print(len(preds_scores))
print(len(proc_texts_scores))	
print(proc_texts_scores)
human_scores,human_scores_mean,model_scores = [],[],[]
for i, text_scores in enumerate(proc_texts_scores):
  sent = set_proc_texts[i]
	human_scores_mean.append(np.mean([int(sc) for sc in proc_texts_scores[sent].split()]))
	model_scores.append(preds_scores[sent])
 
texts_human_scores_mean_01 = (human_scores_mean-np.min(human_scores_mean))/(max(human_scores_mean)-min(human_scores_mean))
spearman_sentavg_posneg, p_value_sentavg_posneg = spearmanr(texts_human_scores_mean_01, model_scores)
kendall_sentavg_posneg, pk_value_sentavg_posneg = kendalltau(texts_human_scores_mean_01, model_scores)
print('The Spearman correlation is {} ({})'.format(spearman_sentavg_posneg,p_value_sentavg_posneg ))
print('The Kendall correlation is {} ({})'.format(kendall_sentavg_posneg,pk_value_sentavg_posneg ))

300
300
{"`` Fine Red Silk , Golden Thread . '' I looked down at the threads that covered my body . It was a simple , simple thing , but it was a beautiful thing . I was sitting on my bed , looking at the sheet I had put on , and I could feel the warmth of the sheets as I looked at the fabric . `` I 'm sorry , I do n't think I can do that . '' `` I know , I know . '' The silk was soft , and it was soft . It felt like a silk pillow , and as I laid down , I felt a warmth that I had never felt before . `` Why do naught but the sheets ? '' I asked , and the fabric was soft again . `` You know , you 're right . '' She said , and she looked at me . `` What ? '' `` You 're not lying . '' My voice was weak , but I knew what she was saying . `` No . '' Her voice was like a whisper , but she was n't . `` It 's not . '' An echo , and a vibration , and then a flash of light . The light was a light , but not a light . It seemed to come from somewhere , and yet it was there . `` Do n't you know what